In [0]:
from pyspark.sql import functions as F


data = [
            (1,['a','b','c'], {'x':1,'y':2},None,[]),
            (2, None,{"m":5},["u"], None),
            (3, ["p"],None,[],["red","green"])
]

schema = 'id INT, arr ARRAY<STRING>, mp MAP<STRING, INT>, tag ARRAY<STRING>, color ARRAY<STRING> '

df = spark.createDataFrame(data,schema)
df.display()

In [0]:
df_arr = df.select("id", F.explode("arr").alias("elem"))
df_arr.show()


In [0]:
df_map = df.select('id',F.explode('mp').alias('key','value'))
df_map.display()

In [0]:
z = F.arrays_zip('arr','color')

out = df.select("id",F.explode(z).alias('z'))

out1 = out.select("id",'z.arr','z.color')
out1.display()

###  SQL equivalents (if you prefer writing SQL)

In [0]:
%python

df.createOrReplaceTempView('df')

In [0]:
%sql
-- Explode array

select * from df;

In [0]:
%sql
select 
id,
a.elem,
c.color,
t.tg,
m.ky,
m.val

from df d
LATERAL VIEW explode_outer(d.arr)   a AS elem
lateral view explode_outer(d.color) c as color
lateral view explode_outer(d.tag) t as tg
lateral view explode_outer(d.mp) m as ky,val
-- Explode

In [0]:

df.select("id", F.posexplode('arr').alias('pos','elem')).display()


In [0]:
%sql

select 
id,
a.pos,
a.elem,
c.pos_color,
c.color,
m.pos_mp,
m.ky,
m.val,
t.pos_tg,
t.tg

 from df d
 lateral view posexplode_outer(d.arr) a as pos,elem
 lateral view posexplode_outer(d.mp) m as pos_mp,ky,val
 lateral view posexplode_outer(d.tag) t as pos_tg,tg
 lateral view posexplode_outer(d.color) c as pos_color, color